In [10]:
import pandas as pd
# import geopandas
from shapely.geometry import Point
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime
%matplotlib inline

Load the data

In [335]:
r1in = pd.read_csv('/Users/maryrosefissinger/Documents/MIT/Classes/IAP_2019/Assignment3/AVLData/SouthMassAve/r1_in.csv')
r1out = pd.read_csv('/Users/maryrosefissinger/Documents/MIT/Classes/IAP_2019/Assignment3/AVLData/SouthMassAve/r1_out.csv')
r71in = pd.read_csv('/Users/maryrosefissinger/Documents/MIT/Classes/IAP_2019/Assignment3/AVLData/MtAuburnSt/r71_in.csv')
r71out = pd.read_csv('/Users/maryrosefissinger/Documents/MIT/Classes/IAP_2019/Assignment3/AVLData/MtAuburnSt/r71_out.csv')
r73in = pd.read_csv('/Users/maryrosefissinger/Documents/MIT/Classes/IAP_2019/Assignment3/AVLData/MtAuburnSt/r73_in.csv')
r73out = pd.read_csv('/Users/maryrosefissinger/Documents/MIT/Classes/IAP_2019/Assignment3/AVLData/MtAuburnSt/r73_out.csv')
r1in.head()

,year,tripdate,prepostm,trip,72,73,75,77
0,2015,2015-09-01 00:00:00,0,42052207,2015-09-01 07:38:03,2015-09-01 07:41:10,2015-09-01 07:46:00,2015-09-01 07:48:24
1,2015,2015-09-01 00:00:00,0,42052376,2015-09-01 07:48:41,2015-09-01 07:51:10,2015-09-01 07:56:10,2015-09-01 07:59:22
2,2015,2015-09-01 00:00:00,0,42051228,2015-09-01 08:03:45,2015-09-01 08:06:45,2015-09-01 08:09:22,2015-09-01 08:11:39
3,2015,2015-09-01 00:00:00,0,42054621,2015-09-01 08:35:23,2015-09-01 08:38:43,2015-09-01 08:44:22,2015-09-01 08:47:58
4,2015,2015-09-01 00:00:00,0,42052202,2015-09-01 09:18:46,2015-09-01 09:22:01,2015-09-01 09:25:59,2015-09-01 09:29:42


In [336]:
# Write functions to get the number of seconds from a timedelta and the hour from a timestamp
f = lambda x: x.seconds
g = lambda x: x.hour

Now we write a function that takes as an input the data we just read in, the route name, and the direction, and returns a dataframe with one row for each subsegment and the entire study area (between the first and final stops) per trip with the year, month, date, time period (eg. 'AM Peak'), prepostm (boolean indicating if it was before or after implementation), segment label, and travel time in seconds. Thus we can run this function on each of the input data and then vertically concatenate them with Route and Direction labels so that all our data is in a single table.

In [337]:
def data_prep(df, route, direction):
    #Rename the columns so the same function will work on all data
    df.columns = ['year','tripdate','prepostm','trip','1','2','3','4']
    
    #Convert necessary columns to easy-to-work-with datetime objects
    df['tripdate'] = pd.to_datetime(df['tripdate'])
    df['1'] = pd.to_datetime(df['1'])
    df['2'] = pd.to_datetime(df['2'])
    df['3'] = pd.to_datetime(df['3'])
    df['4'] = pd.to_datetime(df['4'])
    
    #Get the travel time in seconds for each of the sub-segments
    df['1_2'] = (df['2'] - df['1']).apply(f)
    df['2_3'] = (df['3'] - df['2']).apply(f)
    df['3_4'] = (df['4'] - df['3']).apply(f)
    df['1_4'] = (df['4'] - df['1']).apply(f) #The entire segment
    
    #Make a column that assigns the proper time of day label to each observation
    df['time_period'] = pd.cut(
        df[['1','2','3','4']].max(axis = 1).apply(g), 
        bins=[-np.inf,11,15, np.inf], labels=['AM Peak','Midday', 'PM Peak'])
    
    #Extract the month number
    df['month'] = df['tripdate'].apply(lambda x: x.month)

    #Unpivot the table so the travel time values don't live in separate columns headed 
    #by sub-segment, but rather all in one column with a separate column labeling the segments
    #This will make it easier to filter in R.
    df_unpiv = pd.melt(df, 
                 id_vars=['year','month','tripdate','time_period','prepostm'],
                 value_vars=['1_2', '2_3','3_4','1_4'])
    
    #Rename the columns resulting from the previous step
    df_unpiv =df_unpiv.rename(columns ={'variable': 'segment', 'value':'travel_time'})
    
    #Filter out outliers
    df_unpiv = df_unpiv[df_unpiv['travel_time'] < 4000.0]
    
    #Make 2 new label columns for each set of data: One that says the Route and the other
    #the direction. This way we can concatenate the data to a single table without losing
    #that information
    final = df_unpiv.assign(Route = route, Direction = direction)
    
    return final

In [338]:
#Now concatenate all the prepped data frames
data = pd.concat([data_prep(r1in, 'Route1','Inbound'), data_prep(r1out, 'Route1','Outbound'),
                 data_prep(r71in, 'Route71','Inbound'), data_prep(r71out, 'Route71','Outbound'),
                 data_prep(r73in, 'Route73','Inbound'), data_prep(r73out, 'Route73','Outbound')])

In [340]:
#Read the data to a csv
data.to_csv('/Users/maryrosefissinger/Documents/MIT/Classes/IAP_2019/Assignment3/full_data.csv')